In [29]:
import pandas as pd
import numpy as np 
from datetime import datetime
import mailbox
import regex as re 
import os 
import boto3 
from dotenv import load_dotenv, find_dotenv

In [30]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
s3_secret_key = os.environ['AWS_SECRET_ACCESS_KEY']
s3_bucket_name = os.environ['JUPYTERHUB_USER']
s3_endpoint_url = os.environ['S3_ENDPOINT_URL']
s3_access_key = os.environ['AWS_ACCESS_KEY_ID']
s3bucket = os.environ['BUCKET']

print(s3_endpoint_url)
print(s3_bucket_name)
s3 = boto3.client('s3','us-east-1', endpoint_url= s3_endpoint_url,
                       aws_access_key_id = s3_access_key,
                       aws_secret_access_key = s3_secret_key)

https://s3.upshift.redhat.com
cdolfi


The "msgs" portion is where the errors come with trying to get the mbox

In [ ]:
obj = s3.get_object(Bucket =s3bucket, Key = 'cdolfi/gzip/2017_10-2018_11.gzip')


In [31]:

f_list = input("Are you looking at dev or user? ")
start_year = input("What year do you want to start with? example 2016: " )
start_month = input("What month do you want to start with? example 01: " )

finish_year = input("What year do you want to end with? example 2016: " )
finish_month = input("What month do you want to end with? example 01: " )
finish_month = str((int(finish_month)%12)+1)

file = "cdolfi/mboxes/" + start_year + "_" + start_month + "-" + finish_year + "_"+  finish_month + ".mbox"
obj = s3.get_object(Bucket =s3bucket, Key = file)
#THIS IS WHERE THE ISSUE COMES
msgs = mailbox.mbox(obj['Body'])

Are you looking at dev or user? dev
What year do you want to start with? example 2016: 2016
What month do you want to start with? example 01: 10
What year do you want to end with? example 2016: 2017
What month do you want to end with? example 01: 10


TypeError: expected str, bytes or os.PathLike object, not StreamingBody

In [13]:
#how to get text from a message in the mbox 
def get_text(msg):
    while msg.is_multipart():
        msg = msg.get_payload()[0]
    return msg.get_payload()

In [1]:
#strip thread text and dates in body text
def strip_thread(text):
    text = text.replace("\r", "")
    lines = text.split("\n")
    lines = [l for l in lines if len(l) > 0]
    lines = [line for line in lines if line[0] != ">"]
    lines = [line for line in lines if line[:3] != "Re:"]
    lines = [line for line in lines if line[:7] != "Subject"]
    lines = [line for line in lines if line[:5] != "From:"]
    lines = [line for line in lines if line[:5] != "Date:"]
    lines = [line for line in lines if "BEGIN PGP SIGNED MESSAGE" not in line]
    lines = [line for line in lines if line[:5] != "Hash:"]
    lines = [line for line in lines if line[:10] != "Version: G"]
    lines = [line for line in lines if "wrote:" not in line]
    lines = [line for line in lines if "wrote :" not in line]
    lines = [line for line in lines if "writes:" not in line]
    lines = [line for line in lines if line[:7] != "Am Mit,"]
    lines = [line for line in lines if line[:7] != "Am Don,"]
    lines = [line for line in lines if line[:7] != "Am Mon,"]
    lines = [line for line in lines if line[:7] != "Quoting"]
    lines = [line for line in lines if line[:10] != "Em Quinta,"]
    lines = [line for line in lines if "said:" not in line]
    lines = [line for line in lines if re.match(".*n (Sun|Mon|Tue|Wed|Thu|Fri|Sat), .. (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec) 20..*", line) == None]
    lines = [line for line in lines if re.match(".*n (Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday) .. (January|February|March|April|May|June|July|August|September|October|November|December) 20..*", line) == None]
    lines = [line for line in lines if re.match(".*n (Sun|Mon|Tue|Wed|Thu|Fri|Sat), (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec) .., 20..*", line) == None]
    lines = [line for line in lines if re.match(".*n (Sun|Mon|Tue|Wed|Thu|Fri|Sat), 20[\d]{2}-[\d]{2}-[\d]{2} at.*", line) == None]
    lines = [line for line in lines if line[-6:] != "said: "]
    lines = [line for line in lines if line[-8:] != "babbled:"]
    lines = [line for line in lines if line[-7:] != "wrot=e:"]
    lines = [line for line in lines if line[-8:] != "A9crit :"]
    lines = [line for line in lines if line[0] != "|"]
    return "\n".join(lines)

In [15]:
#build 2d list of email data with thread text and dates removed
def build_list(mbox):
    chart = [] 
    for msg in mbox: 
        clean_body = strip_thread(get_text(msg))
        entry = [clean_body, msg["Date"][:-9], msg["From"], msg["Subject"]]
        chart.append(entry)
    return chart
    

In [16]:
#format for CSV, clean special characters, and remove extranous emails 
def pandas_clean(chart):
    emails = pd.DataFrame(chart, columns = ["Body", 'Date', "From", "Subject"]) 
    emails['Body'].replace(to_replace=[r"\n", "\n",], value= " ", regex=True, inplace=True)
    emails['Body'].replace(to_replace=[r"\'", "\'",">",'<',"= ", "-", "http\S+" ], value="", regex=True, inplace=True)
    emails['Body'].replace(to_replace=[ r"\\\s+", "\\\s+" , "="], value= "", regex=True, inplace=True)
    emails['Body'].replace(to_replace= [ "   ", "  "], value= " ", regex=True, inplace=True)
    emails['Body'].replace(to_replace= ["_","3D"], value= "", regex=True, inplace=True)
    emails['Body'].replace(to_replace= [ "   ", "  "], value= " ", regex=True, inplace=True)
    emails['Body'].replace(to_replace= [ "   ", "  "], value= " ", regex=True, inplace=True)
    
    emails.drop(emails.index[emails['Body'] == ''], inplace = True)
    emails.drop(emails.index[emails['Body'] == ' '], inplace = True)
    emails.dropna(subset=['Body'], inplace=True)
    emails['location'] = emails['Body'].str.find('Missing expected images')
    
    emails.drop(emails.index[emails['location'] == 0], inplace = True)
    emails.drop('location' ,axis=1, inplace=True )
    emails['location'] = emails['Body'].str.find('OLD: Fedora')
    emails.drop(emails.index[emails['location'] == 0], inplace = True)
    emails.drop('location' ,axis=1, inplace=True )
    
    emails['datetime'] = pd.to_datetime(emails['Date'], format='%a, %d %b %Y %H:%M')
    emails.sort_values(by= 'datetime', inplace = True)
    
    emails = emails.reset_index()
    emails.drop('index',axis=1, inplace=True )
    return emails
    

In [17]:
#preforming function on mbox 

mbox_info = build_list(msgs)
data = pandas_clean(mbox_info)

#updated this to save to bucket with naming conventions depending on the mbox that are retrieved
#updated_email.to_csv('/opt/app-root/src/data/user_clean2.csv')

In [ ]:
data.to_csv('/opt/app-root/src/data/temp_clean.csv')
s3_location = "cdolfi/cleaned_" +f_list + "/" + start_year + "_" + start_month + "-" + finish_year + "_"+  finish_month + ".mbox"
s3.upload_file(Filename='/opt/app-root/src/data/temp_clean.csv',Bucket=s3bucket, Key=s3_location)